# Simulation demo notebook

In this notebook we walk through how to run a simulation with a very simple intelligence layer and interactions. Please see the README for more information about the simulation configuration script and the intelligence layer (we will not go into detail about the intelligence layer here). Here we will be using the files in `template/example`, and going through main processes that are called when `patient_abm.simulation.run.simulate` is exexcuted (which is the function called by the CLI command `patient_abm simulation run`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

PATIENT_ABM_DIR = Path(os.getcwd()).parent
PATIENT_ABM_DIR

In [ ]:
%env PATIENT_ABM_DIR={str(PATIENT_ABM_DIR)}

In [ ]:
import copy
import datetime
import json
import logging
import pprint
import uuid
from dataclasses import asdict

from IPython.display import Image

from patient_abm.agent.patient import (
    PatientAgent, PatientRecordEntry, wrap_fhir_resource
)
from patient_abm.log import configure_logger
from patient_abm.simulation.initialize import initialize
from patient_abm.simulation.run import simulate, run_patient_simulation
from patient_abm.utils import string_to_datetime, datetime_to_string

# Load config

In [ ]:
config_path = "../template/example/config.json"

In [ ]:
with open(config_path, "r") as f:
    config = json.load(f)

In [ ]:
pprint.pprint(config)

# Initialise simulation

Initialising a simulation involves:
- validating the config object (see the `patient_abm.simulation.template.parse_config` function)
- loading the patient and agent objects
- loading the `intelligence` function handle
- creating the interaction name to function handle dictionary `interaction_mapper`
- validating and loading all other simulation parameters

### A note on the patient and environment agent attributes

In the `patient-agent` notebook, we demonstrated how to load the patient agent with comorbidities. The `initialize` function below is not suitable if the patient `conditions` attribute is set. Currently, only patient attributes with simpler data structures are checked in the validation process. For the patient these are given in the `PatientConfig` model in `patient_abm.simulation.template`:

``` 
patient_id
gender
birth_date
start_time
name
kwargs
```

We recommend building out the validation module `patient_abm.simulation.template` to handle all patient inputs. And the same goes for the environment.


Need to update some paths in the config so that they can be located from this notebook

In [ ]:
config['intelligence_dir'] = str(PATIENT_ABM_DIR / config['intelligence_dir'])
config['save_dir'] = str(PATIENT_ABM_DIR / config['save_dir'])


In [ ]:
(
    patients,
    environments,
    interaction_mapper,
    intelligence,
    initial_environment_ids,
    stopping_condition_kwargs,
    log_every,
    log_intermediate,
    hard_stop,
    log_patient_record,
    patient_record_duplicate_action,
    save_dir,
    fhir_server_url,
) = initialize(config)

In [ ]:
interaction_mapper

In [ ]:
intelligence

In [ ]:
stopping_condition_kwargs

In [ ]:
log_every, hard_stop, log_patient_record, str(save_dir)

In [ ]:
initial_environment_ids

# Simulate patients one after another 

Make `len(patients)` copies of the environments to use in every patient simulation

In [ ]:
environments_copies = [
    copy.deepcopy(environments) for _ in range(len(patients))
]

We simply loop over the patients, running a simulation for each one by calling `run_patient_simulation`. Inside `run_patient_simulation` (which is inside the `patient_abm.simulation.run` module) there is a while loop which propagates the simulation. At every step of the while loop:

- for every patient, a `simulation_id` is generated, and the outputs for that patient are saved in `save_dir / <simulation_id>`, where `save_dir` is specified in the `config`
- the `intelligence` function is called on the `patient` and an `environment` (starting from the `initial_environment_id`
- the patient and envinronment `update` methods are called:
    - the new patient record entries that are generated by the `intelligence` get added to the existing patient record, and the patient conditions, medications and actions tables are updated
    - the environment `patient_interaction_history` attribute is updated
- the simulation is logged (if at the correct `log_every` step) to `main.log` and `patient.log`
- check for stopping conditions or patient death:
    - if either of these are true, then the simulation terminates
    - if not, then the next environment is selected, and the loop continues
- once the loop terminates, informa the patients and environments are saved as tar files, and a FHIR bundle is generated from the patient record entry.

### An aside: Updating the patient and agent in the intelligence layer

Since every step of the simulation calls the patient and environment update methods, it is not necessary to apply these processes in the intelligence layer. There are however cases when more custom updates are required, for instance, modifying an existing entry in the patient health record, or changing other attributes. See the `death` interaction function in `patient_abm.intelligence.interactions.default` for an example.

Below runs a simple simulation, normally the simulation would not print so much information, this is simply because we have written the intellignece layer in this way for demonstration purposes.

In [ ]:
patient_id_to_agents = {}

for patient, environments, initial_environment_id in zip(
    patients, environments_copies, initial_environment_ids
):

    patient, environments, simulation_id = run_patient_simulation(
        None,
        patient,
        environments,
        interaction_mapper,
        intelligence,
        initial_environment_id,
        stopping_condition_kwargs,
        log_every,
        log_intermediate,
        hard_stop,
        log_patient_record,
        patient_record_duplicate_action,
        save_dir,
        fhir_server_url,
    )
    patient_id_to_agents[patient.patient_id] = {
        "patient": patient,
        "envinroments": environments,
        "simulation_id": simulation_id,
    }


# Look at logs

Let's look at the logs for just a single patient. There are two loggers in the simulation - one for patient specific data and the other for more general simulation data.

In [ ]:
simulation_id = patient_id_to_agents[0]["simulation_id"]

In [ ]:
patient_log_path = Path(config["save_dir"]) / simulation_id / "patient.log"
main_log_path = Path(config["save_dir"]) / simulation_id / "main.log"

The log files are written using the python `pythonjsonlogger` library so they are jsonlines files, which makes them really convenient for parsing as the log data can be read in as a list of dictionaries

In [ ]:
patient_log = []
with open(patient_log_path, "r") as f:
    for line in f:
        patient_log.append(json.loads(line))
        
main_log = []
with open(main_log_path, "r") as f:
    for line in f:
        main_log.append(json.loads(line))

The patient log file contains the usual log file info, plus patient-specific data in the 'patient_state' and 'patient_record_entry' fields. The latter can be omitted if `log_patient_record` is set to false in the `config`. The full patient record is stored in the saved patient object, so nothing data is lost.

In [ ]:
patient_log[-1]

The main logger contains simulation specifics.

In [ ]:
main_log[-1]

# Load saved agents

Above we stored the simulated patient and environment objects in a dictionary `patient_id_to_agents`, however when running the simulation via the CLI, we will have to load the saved agent objects. Let's do that here for patient with patient_id=0

In [ ]:
from patient_abm.agent.patient import PatientAgent
from patient_abm.agent.environment import (
    EnvironmentAgent, GPEnvironmentAgent, AandEEnvironmentAgent
)

In [ ]:
environment_types = [x["type"] for x in config["environments"]]

In [ ]:
def load_environment(path, environment_type):
    if environment_type == "a_and_e":
        environment_class = AandEEnvironmentAgent
    elif environment_type == "gp":
        environment_class = GPEnvironmentAgent
    else:
        environment_class = EnvironmentAgent
    
    return environment_class.load(path)

In [ ]:
patient_id = 0

simulation_id = patient_id_to_agents[patient_id]["simulation_id"]

_patient = PatientAgent.load(
    Path(config["save_dir"]) / simulation_id / "agents" / f"patient_{patient_id}.tar"
)
_environments = [
    load_environment(
        Path(config["save_dir"]) / simulation_id / "agents" / f"environment_{i}.tar",
        environment_types[i]
    )
    for i in [0,1]
]

The record is stored in the patient

In [ ]:
for entry in _patient.record:
    pprint.pprint(asdict(entry))

Was the patient alive at the end of the simulation?

In [ ]:
_patient.alive

Were any conditions added?

In [ ]:
_patient.conditions

# The patient pathway

We can also reconstruct the patient pathway. There are many ways to do this, below we show a simple version

In [ ]:
patient_pathway = [
    (
        datetime_to_string(entry.patient_time),
        environments[entry.environment_id].name,
        entry.interactions,
    )
    for entry in _patient.record
    if entry.environment_id not in [-1, None]
]

In [ ]:
len(patient_pathway)

In [ ]:
patient_pathway

As well as seeing the pathway from the patient's point of view, each environment also holds a history of patient visits in its `patient_interaction_history` attribute, whihc is a dictionary where each key corresponds to the patient_id, and the values visit logs.

In [ ]:
_environments[0].patient_interaction_history

In [ ]:
_environments[1].patient_interaction_history

# The FHIR health record

The simulation also generates a FHIR health record (a Bundle resource type) from the patient record. We can view the bundle for the patients

In [ ]:
from patient_abm.data_handler.fhir import FHIRHandler, HAPI_FHIR_SERVER_4

In [ ]:
fhir_handler = FHIRHandler()

In [ ]:
patient_id_to_bundles = {}
for patient_id, vals in patient_id_to_agents.items():
    fhir_path = Path(config["save_dir"]) / vals["simulation_id"] / "fhir" / "bundle.json"
    patient_id_to_bundles[patient_id] = fhir_handler.load(fhir_path, "Bundle", validate=False)

In [ ]:
bundle_0 = patient_id_to_bundles[0]
bundle_0

In [ ]:
bundle_1 = patient_id_to_bundles[1]
bundle_1

We can also validate these bundles using the HAPI FHIR server

In [ ]:
fhir_handler.validate(bundle_0, server_url=HAPI_FHIR_SERVER_4)

In [ ]:
fhir_handler.validate(bundle_1, server_url=HAPI_FHIR_SERVER_4)

# Initialising agents with attributes written in a file

When we initialised the agents from the config script above, the agent attributes were written directly in the config script. Instead, we can supply a path to a JSON or CSV file contaning the same information.

In [ ]:
from patient_abm import PATIENT_ABM_DIR
from patient_abm.simulation.template import parse_config
from patient_abm.data_handler.base import DataHandler

TEST_DATA_DIR = PATIENT_ABM_DIR / "tests" / "data"

In [ ]:
data_handler = DataHandler()

### Example 1: Patient and environment data stored in JSON

Let's look at the contents of the JSON, it is a list of dictionaries

In [ ]:
patients_json = data_handler.load(TEST_DATA_DIR / "patients_config.json")
environments_json = data_handler.load(TEST_DATA_DIR / "environments_config.json")

In [ ]:
patients_json

In [ ]:
environments_json

And we modify the config script so that `patients` and `environments` point to these JSON file paths

In [ ]:
config["patients"] = str(TEST_DATA_DIR / "patients_config.json")
config["environments"] = str(TEST_DATA_DIR / "environments_config.json")

In [ ]:
config

Before this is passed to the initialize function, it must be parder

In [ ]:
parsed_config = parse_config(config)

In [ ]:
parsed_config

Then we can proceed as before

In [ ]:
(
    patients,
    environments,
    interaction_mapper,
    intelligence,
    initial_environment_ids,
    stopping_condition_kwargs,
    log_every,
    log_intermediate,
    hard_stop,
    log_patient_record,
    patient_record_duplicate_action,
    save_dir,
    fhir_server_url,
) = initialize(parsed_config)

### Example 2: Patient and environment data stored in CSV

Let's look at the contents of the CSV

In [ ]:
patients_csv = data_handler.load(TEST_DATA_DIR / "patients_config.csv")
environments_csv = data_handler.load(TEST_DATA_DIR / "environments_config.csv", index_col=0)

In [ ]:
patients_csv

In [ ]:
environments_csv

These CSV paths can also be used in the config

In [ ]:
config["patients"] = str(TEST_DATA_DIR / "patients_config.csv")
config["environments"] = str(TEST_DATA_DIR / "environments_config.csv")

In [ ]:
config

And we can parse the config and proceed as before

In [ ]:
parsed_config = parse_config(config)

In [ ]:
parsed_config

Although methods to parse inputs from both JSON and CSV are implemented, the JSON version is strongly preferred because it represent the true datatypes better. For example, some attributes like enviroments `interactions` are nested objects (a list of strings). This is represented as a string in the CSV and has to be converted to a list of strings during the parsing stage (see `patient_abm.simulation.template.parse_agents_config_from_file`) which could become complex to track in future.

In any case, we do recommend developing the functionality `patient_abm.simulation.template` so that more agent attributes can be parsed and validated.